In [1]:
import torch
from transformers import AutoModel, AutoTokenizer 
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

# For transformers v4.x+: 
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
tweets = pd.read_csv('../bertweet_embeddings/all_china_full_nort.csv')
tweets = tweets.dropna() # some rows come in as blank so they need to be dropped
train, test = train_test_split(tweets, test_size=0.2)

In [4]:
# this cell tests for invalid or blank text entries and prints their index if one comes up

# for idx, tweet in enumerate(text):
#     try: 
#         tokenizer(tweet, padding='max_length', max_length=150, return_tensors="pt")
#     except:
#         print(idx, tweet)

In [5]:
# reading in tensors from file

embeddings = torch.Tensor()

for i in range(39):
    filename = "../bertweet_embeddings/embeddings/all_china_embedding_" + str(i*1000) + ".pt"
    embeddings = torch.cat((embeddings, torch.load(filename)))
    
print(embeddings.shape, tweets.shape) # these should be in agreement

torch.Size([38733, 768]) (38733, 3)


In [6]:
X_train = embeddings[train.index].detach().numpy()
y_train = train['is_ccp']

print(X_train.shape, y_train.shape)

(30986, 768) (30986,)


In [7]:
model = keras.Sequential()

#model.add(keras.layers.Normalization())

model.add(keras.layers.Dense(16, 
                             activation = 'relu'))

model.add(keras.layers.Dense(1, activation = 'sigmoid'))

#optimizer = keras.optimizers.Adam(clipvalue=1, learning_rate=0.0001)

model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=['accuracy']) 

In [8]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
969/969 [==============================] - 3s 1ms/step - loss: 0.3359 - accuracy: 0.8653
Epoch 2/20
969/969 [==============================] - 1s 1ms/step - loss: 0.2428 - accuracy: 0.9077
Epoch 3/20
969/969 [==============================] - 1s 1ms/step - loss: 0.2202 - accuracy: 0.9175
Epoch 4/20
969/969 [==============================] - 1s 1ms/step - loss: 0.2098 - accuracy: 0.9219
Epoch 5/20
969/969 [==============================] - 1s 1ms/step - loss: 0.2044 - accuracy: 0.9249
Epoch 6/20
969/969 [==============================] - 1s 1ms/step - loss: 0.1989 - accuracy: 0.9269
Epoch 7/20
969/969 [==============================] - 1s 1ms/step - loss: 0.1941 - accuracy: 0.9279
Epoch 8/20
969/969 [==============================] - 1s 1ms/step - loss: 0.1924 - accuracy: 0.9281
Epoch 9/20
969/969 [==============================] - 1s 1ms/step - loss: 0.1873 - accuracy: 0.9309
Epoch 10/20
969/969 [==============================] - 1s 1ms/step - loss: 0.1840 - accuracy: 0.9317

# Testing

In [9]:
X_test = embeddings[test.index].detach().numpy()
y_test = test['is_ccp']

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
y_pred_bool = np.where(y_pred >= 0.5, 1, 0).ravel() #DIY function to round outputs to 0 or 1

print(classification_report(y_test, y_pred_bool))
print(confusion_matrix(y_test, y_pred_bool))

              precision    recall  f1-score   support

         0.0       0.98      0.84      0.90      3836
         1.0       0.86      0.98      0.92      3911

    accuracy                           0.91      7747
   macro avg       0.92      0.91      0.91      7747
weighted avg       0.92      0.91      0.91      7747

[[3209  627]
 [  68 3843]]


In [11]:
model.evaluate(X_test, y_test)   

243/243 [==============================] - 0s 855us/step - loss: 0.2079 - accuracy: 0.9103


[0.20789873600006104, 0.9102878570556641]

In [12]:
# one line tester

line = "xinjiang is a slavery concentration camp"
line_token = tokenizer(line, padding='max_length', max_length=130, return_tensors="pt")
line_embed = bertweet(**line_token)

model.predict(line_embed.pooler_output.detach().numpy())

array([[0.9234344]], dtype=float32)

# Test on Pro China set

In [13]:
tweets_pro = pd.read_csv('../bertweet_embeddings/pro_china_full_nort.csv')
tweets_pro = tweets_pro.dropna() # some rows come in as blank so they need to be dropped
xxx, test_pro = train_test_split(tweets_pro, test_size=0.5, random_state=0)

In [14]:
embeddings_pro = torch.Tensor()

for i in range(39):
    filename = "../bertweet_embeddings/embeddings/pro_china_embedding_" + str(i*1000) + ".pt"
    embeddings_pro = torch.cat((embeddings_pro, torch.load(filename)))
    
print(embeddings_pro.shape, tweets_pro.shape)

torch.Size([38734, 768]) (38733, 3)


In [1]:
X_test_pro = embeddings_pro[test_pro.index].detach().numpy()
y_test_pro = test_pro['is_ccp']

from sklearn.metrics import classification_report, confusion_matrix

y_pred_pro = model.predict(X_test_pro)
y_pred_pro_bool = np.where(y_pred_pro >= 0.5, 1, 0).ravel() #DIY function to round outputs to 0 or 1

print(classification_report(y_test_pro, y_pred_pro_bool))
print(confusion_matrix(y_test_pro, y_pred_pro_bool))

model.evaluate(X_test_pro, y_test_pro)   

NameError: name 'embeddings_pro' is not defined